<a href="https://colab.research.google.com/github/BoyPlankton/public_notebooks/blob/master/2019_FiveThirtyEight_NFL_Forecast_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q trueskill
!pip install --upgrade -q gspread
!pip install --upgrade -q gspread-dataframe

In [0]:
import pandas as pd
import io
import requests

s=requests.get("https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv").content
data=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [0]:
import numpy as np

team_abbr = np.append(data.team1.unique(), data.team2.unique())

team_abbr = np.unique([x for x in team_abbr if isinstance(x, str)])

In [0]:
from trueskill import Rating, rate_1vs1, BETA, global_env
import math
import itertools

teams = {}

for x in team_abbr:
  teams[x] = Rating()

# from the example at https://trueskill.org
def win_probability(team1, team2):
  delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
  sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
  size = len(team1) + len(team2)
  denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
  ts = global_env()
  return ts.cdf(delta_mu / denom)

def eval(x):
  wp_h = win_probability([teams[x["team1"]]], [teams[x["team2"]]])
  wp_v = win_probability([teams[x["team2"]]], [teams[x["team1"]]])

  if wp_h > wp_v:
    return [x["team1"], wp_h]
  elif wp_v > wp_h:
    return [x["team2"], wp_v]

  return ["tie", wp_h]

def upd_ratings(x):
  if x["score1"] > x["score2"]:
    n1, n2 = rate_1vs1(teams[x["team1"]], teams[x["team2"]])
    win = x["team1"]
  elif x["score2"] > x["score1"]:
    n2, n1 = rate_1vs1(teams[x["team2"]], teams[x["team1"]])
    win = x["team2"]
  else:
    n1, n2 = rate_1vs1(teams[x["team1"]], teams[x["team2"]], drawn=True)

  teams[x["team1"]] = n1
  teams[x["team2"]] = n2

def upd_eval(x):
  y = eval(x)

  upd_ratings(x)

  return y

data['wp'] = data[data['date'] < '2019-11-14'].apply(lambda x: upd_eval(x), axis=1)

In [0]:
data['wp'] = data[(data['date'] >= '2019-11-14') & (data['date'] <= '2019-11-18')].apply(lambda x: eval(x), axis=1)

In [19]:
data[(data['date'] >= '2019-11-14') & (data['date'] <= '2019-11-18')][['team1','team2','elo_prob1','elo_prob2','score1','score2','wp']]

,team1,team2,elo_prob1,elo_prob2,score1,score2,wp
148,CLE,PIT,0.398335,0.601665,NaN,NaN,"[PIT, 0.6688334057753781]"
149,MIN,DEN,0.800677,0.199323,NaN,NaN,"[MIN, 0.8390304437046163]"
150,BAL,HOU,0.679152,0.320848,NaN,NaN,"[BAL, 0.6595148863250907]"
151,WSH,NYJ,0.604104,0.395896,NaN,NaN,"[NYJ, 0.5631860978344945]"
152,CAR,ATL,0.688945,0.311055,NaN,NaN,"[CAR, 0.8136894613399841]"
153,IND,JAX,0.708369,0.291631,NaN,NaN,"[IND, 0.6805307890550811]"
154,DET,DAL,0.447757,0.552243,NaN,NaN,"[DAL, 0.559488708004384]"
155,TB,NO,0.348555,0.651445,NaN,NaN,"[NO, 0.845648241939469]"
156,MIA,BUF,0.428276,0.571724,NaN,NaN,"[BUF, 0.7365510032755871]"
157,SF,ARI,0.847265,0.152735,NaN,NaN,"[SF, 0.9480295861469703]"


In [0]:
teams

{'ARI': trueskill.Rating(mu=22.416, sigma=3.514),
 'ATL': trueskill.Rating(mu=15.976, sigma=3.648),
 'BAL': trueskill.Rating(mu=32.306, sigma=3.554),
 'BUF': trueskill.Rating(mu=27.110, sigma=3.978),
 'CAR': trueskill.Rating(mu=28.639, sigma=3.532),
 'CHI': trueskill.Rating(mu=21.077, sigma=3.429),
 'CIN': trueskill.Rating(mu=11.239, sigma=4.262),
 'CLE': trueskill.Rating(mu=21.660, sigma=3.699),
 'DAL': trueskill.Rating(mu=25.084, sigma=3.538),
 'DEN': trueskill.Rating(mu=22.593, sigma=3.447),
 'DET': trueskill.Rating(mu=24.968, sigma=3.427),
 'GB': trueskill.Rating(mu=30.175, sigma=3.190),
 'HOU': trueskill.Rating(mu=29.218, sigma=3.186),
 'IND': trueskill.Rating(mu=28.995, sigma=3.323),
 'JAX': trueskill.Rating(mu=22.276, sigma=3.553),
 'KC': trueskill.Rating(mu=30.405, sigma=3.189),
 'LAC': trueskill.Rating(mu=21.967, sigma=2.915),
 'LAR': trueskill.Rating(mu=25.857, sigma=3.858),
 'MIA': trueskill.Rating(mu=15.116, sigma=3.927),
 'MIN': trueskill.Rating(mu=29.265, sigma=3.349),
 '